<a href="https://colab.research.google.com/github/NastasiaMazur/Finance-Sentiment-Analysis/blob/main/distilbert_base_uncased_FinanceInc_auditor_sentiment__UPDATED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downstream Task: Sentiment Analysis

Model: **distilbert-base-uncased**

Dataset: **FinanceInc/auditor_sentiment**

# 1. Activate GPU and Install Dependencies

In [1]:
#check if GPU is available
import torch
torch.cuda.is_available()

True

In [2]:
# Install required libraries
!pip install datasets
!pip install transformers
!pip install bertviz transformers
!pip install transformers[torch]
#!apt-get install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-m

In [3]:
# Conntext Google Drive so data can be stored there
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 2. Preprocess data

In [4]:
# Load data
from datasets import load_dataset
finance_dataset = load_dataset("FinanceInc/auditor_sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3877 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/969 [00:00<?, ? examples/s]

In [5]:
# Create a smaller training dataset for faster training times
from datasets import DatasetDict

small_finance_dataset = DatasetDict(
    train=finance_dataset['train'].shuffle(seed=24).select(range(500)), # for training
    val=finance_dataset['train'].shuffle(seed=24).select(range(500, 600)), #  for validation
    test=finance_dataset['train'].shuffle(seed=24).select(range(600, 700)) # for testing
)

In [6]:
small_finance_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 500
    })
    val: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
})

In [7]:
small_finance_dataset['train'][:5]

{'sentence': ['FCC Chairman Kevin Martin said that fair play required extending the same deregulatory rules to the digital subscriber lines that telecom providers use for broadband networks .',
  'Metso Foundries Jyvaskyla Oy will discontinue production on this line by 30 September 2008 , the company said .',
  'Finnish business software group AffectoGenimap Oyj said its net profit halved to 1.2 mln euro ( $ 1.5 mln ) in the first nine months of 2006 from 2.2 mln euro ( $ 2.8 mln ) in the same period of 2005 .',
  'Finnish financial software developer Basware Oyj said today it will provide its invoice automation ( IA ) solution to an unnamed major retail company in the USA in a deal , worth more than EUR300 ,000 .',
  'According to the company , a decision in the issue will be made in the summer of 2010 , at the earliest , and in the summer of 2011 , at the latest .'],
 'label': [1, 1, 0, 2, 1]}

In [8]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print(tokenizer)
print(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
30522


In [9]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

small_tokenized_dataset = small_finance_dataset.map(tokenize_function, batched=True, batch_size=16)
small_tokenized_dataset = small_tokenized_dataset.remove_columns(["sentence"])
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
small_tokenized_dataset['train'][0:2]


{'labels': tensor([1, 1]),
 'input_ids': tensor([[  101, 14420,  3472,  4901,  3235,  2056,  2008,  4189,  2377,  3223,
           8402,  1996,  2168,  4315, 13910, 20350,  2100,  3513,  2000,  1996,
           3617,  4942, 29234,  2099,  3210,  2008, 18126, 11670,  2224,  2005,
          19595,  6125,  1012,   102,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0],
         [  101, 15253,  2080,  2179,  5134,  1046,  2100, 12044,  4801,  2721,
           1051,  2100,  2097, 12532, 16778, 11231,  2063,  2537,  2006,  2023,
           2240,  2011,  2382,  2244,  2263,  1010,  1996,  2194,  2056,  1012,
            102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,    

In [11]:
import torch
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_tokenized_dataset['train'], batch_size=16)
eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=16)

# 3. Training the model

In [12]:

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import pipeline
from tqdm.notebook import tqdm

In [13]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
num_epochs = 3
num_training_steps = 3 * len(train_dataloader)
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [17]:
from datasets import load_dataset, DatasetDict

small_finance_dataset = DatasetDict(
    train=finance_dataset['train'].shuffle(seed=24).select(range(300)),
    val=finance_dataset['train'].shuffle(seed=24).select(range(300, 400)),
    test=finance_dataset['train'].shuffle(seed=24).select(range(400, 500))
)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

small_tokenized_dataset = small_finance_dataset.map(tokenize_function, batched=True, batch_size=16)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Uncomment if you want to log in to your Hugging Face account:

In [ ]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
#from huggingface_hub import notebook_login

#notebook_login()

In [18]:
# Define a new Trainer with all the objects we constructed so far
import numpy as np
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

arguments = TrainingArguments(
    output_dir="store_the_checkpoints_distilbert_3",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    lr_scheduler_type='cosine',
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    seed=224
    #push_to_hub=True, # uncomment to push into hugginhface hub
)


early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0
)


optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.867337,0.630000
2,No log,0.790891,0.680000
3,No log,0.708143,0.690000
4,No log,0.612837,0.710000
5,No log,0.561163,0.740000
6,No log,0.505782,0.740000
7,No log,0.493214,0.760000
8,No log,0.495447,0.730000
9,No log,0.486538,0.770000
10,No log,0.485437,0.770000


TrainOutput(global_step=190, training_loss=0.5071853236148232, metrics={'train_runtime': 78.4993, 'train_samples_per_second': 38.217, 'train_steps_per_second': 2.42, 'total_flos': 56117502513792.0, 'train_loss': 0.5071853236148232, 'epoch': 10.0})

In [20]:
results = trainer.predict(small_tokenized_dataset['val'])
print(results)

PredictionOutput(predictions=array([[-2.0119543 ,  0.82507575,  0.82472587],
       [-0.76417845, -1.0299691 ,  1.5050212 ],
       [-1.9252287 ,  2.1525643 , -0.7914759 ],
       [-1.9548198 ,  2.1257436 , -0.78917104],
       [-1.920791  ,  2.2720056 , -0.9773578 ],
       [-1.2838897 ,  0.44202498,  0.5166941 ],
       [-0.67934775,  1.4241952 , -1.3998218 ],
       [-1.5699657 ,  1.2605628 , -0.2760292 ],
       [-1.6161832 ,  2.1358771 , -1.0464462 ],
       [-1.6677535 ,  2.1006432 , -0.9101729 ],
       [-2.00368   ,  2.055775  , -0.587426  ],
       [-1.9088746 ,  2.3489268 , -0.90510255],
       [-1.6353399 ,  1.5073102 , -0.3126022 ],
       [-1.7774292 ,  0.33237404,  1.1524324 ],
       [-1.4591066 ,  0.44248304,  0.5799592 ],
       [-1.8886943 ,  2.4172187 , -1.0114026 ],
       [-1.590177  ,  2.381789  , -1.2686024 ],
       [ 1.4401168 , -1.2501501 , -0.23521811],
       [-1.8486938 ,  0.5114212 ,  0.9690472 ],
       [ 0.3874727 , -1.3356103 ,  0.71025556],
       [-2.

In [22]:
test_str = "The company's strong quarterly earnings report resulted in a surge in stock prices, reflecting investor confidence in its future prospects."

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("store_the_checkpoints_distilbert_3/checkpoint-190") # pass checkpoint to the model
model_inputs = tokenizer(test_str, return_tensors="pt")

prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE", "NEUTRAL"][prediction])

NEUTRAL


In [23]:
results = trainer.predict(small_tokenized_dataset['test'])
print(results)

PredictionOutput(predictions=array([[-1.6865975 ,  2.287389  , -1.1197214 ],
       [-1.6907045 ,  2.3500211 , -1.1866944 ],
       [-1.7152182 ,  0.39457008,  1.0395966 ],
       [-1.7430782 ,  1.1333718 ,  0.0923994 ],
       [-1.6730635 ,  2.2822056 , -1.0225914 ],
       [-0.43812448,  0.6705169 , -0.62580425],
       [-1.7125256 ,  2.367878  , -1.2318171 ],
       [ 1.0389968 , -1.4102434 ,  0.35678628],
       [-0.88922393, -0.83461285,  1.4444058 ],
       [-1.9900331 ,  1.4520876 , -0.10188285],
       [-1.6203183 ,  2.2917793 , -1.0757321 ],
       [ 0.6744091 , -0.250595  , -0.96117455],
       [-1.3571918 , -0.01990249,  0.79013115],
       [-0.72609246, -0.98811156,  1.4969081 ],
       [-1.7373402 ,  2.3851216 , -1.0900558 ],
       [-1.3018802 ,  1.5421008 , -0.84085095],
       [-2.1647918 ,  1.5564442 ,  0.21429843],
       [-1.7425576 ,  2.0666568 , -1.0202616 ],
       [-1.4279201 ,  1.6617621 , -0.80862856],
       [ 0.08915505, -0.01550037, -0.6142642 ],
       [-1.

In [24]:
small_finance_dataset['test'][5]

{'sentence': 'The company will also shut one paper machine in Finland and one in Austria , as well as two label paper machines in Finland for up to 10 months , Helsinki-based UPM said yesterday .',
 'label': 1}

In [26]:
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("store_the_checkpoints_distilbert_3/checkpoint-190")

model_inputs = tokenizer(small_tokenized_dataset['test']['sentence'], padding=True, truncation=True, return_tensors='pt')

outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)

In [27]:
import os
import torch
from torch.utils.tensorboard import SummaryWriter

path = "/content/drive/MyDrive/results_vis_distilbert_2"
layer = 0

if not os.path.exists(path):
    os.mkdir(path)

while layer in range(len(outputs['hidden_states'])):
    if not os.path.exists(path+'/layer_' + str(layer)):
        os.mkdir(path+'/layer_' + str(layer))

    tensors = []
    labels = []

    for example in range(len(outputs['hidden_states'][layer])):
        sp_token_position = 0
        for token in model_inputs['input_ids'][example]:
            if token != 0:
                sp_token_position += 1
            else:
                tensor = outputs['hidden_states'][layer][example][sp_token_position]
                tensors.append(tensor)
                label = [small_tokenized_dataset['test']['sentence'][example], str(small_tokenized_dataset['test']['label'][example])]
                labels.append(label)
                break

    #print("Length of tensors:", len(tensors))
    #print("Length of labels:", len(labels))

    global_step = layer  # Set global_step to the current layer number
    writer = SummaryWriter(path+'/layer_' + str(layer))
    writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Sentence','Emotion'], global_step=global_step)

    layer += 1


 TensorFlow Embedding Projector [API](https://projector.tensorflow.org/).

____________________



To avoid AssertionError:(#labels should equal with #data points) and ensure that labels are extracted consistently with the corresponding tensors ensure that you're extracting labels consistently with how you're extracting tensors.

To avoid a warning: (Embedding dir exists, did you set global_step for add_embedding()? ) add "global_step = layer"

# 4. Analyzing new data with the model

You can also use pipeline (uncomment a cell with huggingface above):

In [ ]:
# Upload the model to the Hub
trainer.push_to_hub()

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="") # Add your saved mode in ""

sentiment_model(["The company's strong quarterly earnings report resulted in a surge in stock prices, reflecting investor confidence in its future prospects."])